In [54]:
library(stringr)
library(quanteda)
library(dplyr)
library(neuralnet)

# Definição de helper functions

Função responsável por remover a acentuação das palavras

In [2]:
rm_accent <- function(str,pattern="all") {
  # Rotinas e funções úteis V 1.0
  # rm.accent - REMOVE ACENTOS DE PALAVRAS
  # Função que tira todos os acentos e pontuações de um vetor de strings.
  # Parâmetros:
  # str - vetor de strings que terão seus acentos retirados.
  # patterns - vetor de strings com um ou mais elementos indicando quais acentos deverão ser retirados.
  #            Para indicar quais acentos deverão ser retirados, um vetor com os símbolos deverão ser passados.
  #            Exemplo: pattern = c("´", "^") retirará os acentos agudos e circunflexos apenas.
  #            Outras palavras aceitas: "all" (retira todos os acentos, que são "´", "'", "^", "~", "¨", "ç")
  if(!is.character(str))
    str <- as.character(str)

  pattern <- unique(pattern)

  if(any(pattern=="Ç"))
    pattern[pattern=="Ç"] <- "ç"

  symbols <- c(
    acute = "áéíóúÁÉÍÓÚýÝ",
    grave = "àèìòùÀÈÌÒÙ",
    circunflex = "âêîôûÂÊÎÔÛ",
    tilde = "ãõÃÕñÑ",
    umlaut = "äëïöüÄËÏÖÜÿ",
    cedil = "çÇ"
  )

  nudeSymbols <- c(
    acute = "aeiouAEIOUyY",
    grave = "aeiouAEIOU",
    circunflex = "aeiouAEIOU",
    tilde = "aoAOnN",
    umlaut = "aeiouAEIOUy",
    cedil = "cC"
  )

  accentTypes <- c("´","'","^","~","¨","ç")

  if(any(c("all","al","a","todos","t","to","tod","todo")%in%pattern)) # opcao retirar todos
    return(chartr(paste(symbols, collapse=""), paste(nudeSymbols, collapse=""), str))

  for(i in which(accentTypes%in%pattern))
    str <- chartr(symbols[i],nudeSymbols[i], str)

  return(str)
}

Função responsável por fazer pré-processamento no texto , fazendo as mudanças:

    * Transformar o texto inteiro para letras minúsculas;
    * Remover símbolo da moeda real (i.e, R$);
    * Remover todos os símbolos que não sejam letras;
    * Remover espaços redundates;

In [3]:
clean_string <- function(string){
    # Lowercase
    temp <- tolower(string)
    # Remove everything that is not a number or letter (may want to keep more 
    # stuff in your actual analyses). 
    temp <- rm_accent(temp)
    temp <- stringr::str_replace_all(temp,"r\\$", " ")
    temp <- stringr::str_replace_all(temp,"[^a-zA-Z\\s]", " ")
    # Shrink down to just one white space
    temp <- stringr::str_replace_all(temp,"[\\s]+", " ")
    # Split it
    return(temp)
}

Transforma o texto em representação vetorial, i.e., uma matriz onde as colunas são termos e cada linha representa um documento, contendo 1 nas colunas cujos termos estão presentes no texto e 0 caso contrário.

In [4]:
vectorize_sequences <- function(sequences, corpus_dfm, top_terms) {
  # Creates an all-zero matrix of shape (length(sequences), dimension)
    results <- matrix(0, nrow = length(sequences), ncol = length(top_terms)) 
    for (i in 1:length(sequences)){
        splitted_sequence <- stringr::str_split(sequences[[i]], " ")[[1]]
        for (j in 1:length(splitted_sequence)){
            term <- dfm_select(corpus_dfm, splitted_sequence[j])
            if(! length(featnames(term))==0){
                term_rank <- match(featnames(term), top_terms)
                if(! is.na(term_rank)){
                    # Sets specific indices of results[i] to 1s
                    results[i, term_rank] <- 1 
                }    
            }
        }
    }
    results
}

# Leitura e processamento dos dados

In [5]:
review_data <- read.csv(file = "./b2w-10k.csv")

In [6]:
head(review_data)

submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state,X,X.1,X.2,X.3,X.4
2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de0243719b15d50623ce,132532965,"Notebook Asus Vivobook Max X541NA-GO472T Intel Celeron Quad Core 4GB 500GB Tela LED 15,6"" Windows - 10 Branco",null,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o único problema com as Americanas é se houver troca ou devolução do produto o consumidor tem problemas com espera.,1958,F,RJ,,,,,
2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a9008da562ead5f5ecd,22562178,Copo Acrílico Com Canudo 500ml Rocie,null,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse lindo copo de acrílico.",1996,M,SC,,,,,
2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f210329b9b71890277fd,113022329,Panela de Pressão Elétrica Philips Walita Daily 5L com Timer,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,"SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANELAS ELÉTRICAS. COSTUMO USAR OUTRA PANELA PARA COZIMENTO DE ARROZ (JAPONESA), MAS LEVA MUITO TEMPO, +/- 50 MINUTOS. NESSA PANELA É MUITO MAIS RÁPIDO, EXATAMENTE 6 MINUTOS. EU RECOMENDO.",1984,M,SP,,,,,
2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863f165ffedb1e8147984,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS DETALHES QUE TÊM!,1985,F,SP,,,,,
2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e72bce7e88850efa0fc,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com Conversor Digital 4 HDMI 2 USB WebOS 3.5 Painel Ips HDR e Magic Mobile Connection",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de parabéns. A smart tv é muito boa, a navegação na internete e pelos aplicativos e excelente, não trava, sem falar da imagem que é de surpreender. recomendo.",1994,M,MG,,,,,
2018-01-01 01:27:23,28b1844e1cd24dd2288b7cafb052a0b46aed53ab28e1c11632e3a2297a64aad8,22562178,Copo Acrílico Com Canudo 500ml Rocie,null,Utilidades Domésticas,"Copos, Taças e Canecas",Produto imperdível,5,Yes,"Excelente produto, por fora em material acrílico super resistente e por dentro em adamantio, faz milagre com qualquer bebida. Sugiro aproveitarem a promoção antes que acabe.",1979,M,PA,,,,,


Separando os campos contendo o texto do review e a coluna com o dado sobre recomendação

In [7]:
filtered_data <- review_data[,c(11,10)]

In [8]:
head(filtered_data)

review_text,recommend_to_a_friend
Estou contente com a compra entrega rápida o único problema com as Americanas é se houver troca ou devolução do produto o consumidor tem problemas com espera.,Yes
"Por apenas R$1994.20,eu consegui comprar esse lindo copo de acrílico.",Yes
"SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANELAS ELÉTRICAS. COSTUMO USAR OUTRA PANELA PARA COZIMENTO DE ARROZ (JAPONESA), MAS LEVA MUITO TEMPO, +/- 50 MINUTOS. NESSA PANELA É MUITO MAIS RÁPIDO, EXATAMENTE 6 MINUTOS. EU RECOMENDO.",Yes
MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS DETALHES QUE TÊM!,Yes
"A entrega foi no prazo, as americanas estão de parabéns. A smart tv é muito boa, a navegação na internete e pelos aplicativos e excelente, não trava, sem falar da imagem que é de surpreender. recomendo.",Yes
"Excelente produto, por fora em material acrílico super resistente e por dentro em adamantio, faz milagre com qualquer bebida. Sugiro aproveitarem a promoção antes que acabe.",Yes


Substituindo o valor `Yes` da coluna por `1`, caso o cliente recomende o produto e `0` caso contrário

In [9]:
filtered_data$score<-ifelse(filtered_data$recommend_to_a_friend=="Yes", 1,0)

Selecionando somente 2k linhas da tabela

In [10]:
filtered_data <- filtered_data[1:2000,c(1,3)]

Limpando o texto utilizando a função `clean_string()`

In [11]:
filtered_data$review_text <- clean_string(filtered_data$review_text)

In [12]:
dim(filtered_data)

[1] 2000    2

In [13]:
head(filtered_data)

review_text,score
estou contente com a compra entrega rapida o unico problema com as americanas e se houver troca ou devolucao do produto o consumidor tem problemas com espera,1
por apenas eu consegui comprar esse lindo copo de acrilico,1
supera em agilidade e praticidade outras panelas eletricas costumo usar outra panela para cozimento de arroz japonesa mas leva muito tempo minutos nessa panela e muito mais rapido exatamente minutos eu recomendo,1
meu filho amou parece de verdade com tantos detalhes que tem,1
a entrega foi no prazo as americanas estao de parabens a smart tv e muito boa a navegacao na internete e pelos aplicativos e excelente nao trava sem falar da imagem que e de surpreender recomendo,1
excelente produto por fora em material acrilico super resistente e por dentro em adamantio faz milagre com qualquer bebida sugiro aproveitarem a promocao antes que acabe,1


# Construção do corpus a partir de todo o texto lido

Concatenando o texto de todas as observações em um único fragmento

In [14]:
df_corpus <- filtered_data %>% summarise(text = paste0(review_text, sep = "", collapse = ". "))

In [15]:
dim(df_corpus)

[1] 1 1

Criando um `corpus` a partir do texto concatenado

In [16]:
meu_corpus <- corpus(df_corpus$text)
summary(meu_corpus)

Text,Types,Tokens,Sentences
text1,4918,44905,1


Criando a matriz de frequência documento-termo 

In [17]:
corpus_dfm <- dfm(meu_corpus, remove_punct = TRUE,
                  remove = quanteda::stopwords("portuguese"))

Selecionando os 2000 termos mais frequentes

In [18]:
sorted_dfm <- dfm_sort(corpus_dfm)[, 1:2000]
top_terms <- featnames(sorted_dfm)

Agora é hora de construir nossa representação numérica do texto. Essa linha deve demorar vários minutos para executar e deve consumir muitos recursos (CPU, RAM). Não recomendo executá-la em uma VM com poucos recursos

In [19]:
#feature_matrix <- vectorize_sequences(filtered_data$review_text, corpus_dfm, top_terms)

Temos uma versão salva da nossa base textual em formato vetorizado no arquivo `feature_matrix.txt`

In [20]:
feature_matrix <- read.table("feature_matrix.txt", header = TRUE)

In [21]:
feature_matrix <- cbind(feature_matrix, filtered_data$score)

Ajustando o nome das colunas do dado lido

In [22]:
names(feature_matrix) <- c(top_terms,"score")

In [23]:
head(feature_matrix)

produto,nao,bom,recomendo,entrega,excelente,qualidade,prazo,bem,chegou,...,capinha,trincar,indicaria,incomodam,pesada,delicado,ferve,curta,distancia,score
1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0,1,0,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Criando rede neural para fazer predições

Separando o dataset em dados de treino e teste

In [39]:
index <- sample(1:nrow(feature_matrix), .8*nrow(feature_matrix))

In [40]:
train_split <- feature_matrix[index,]
test_split <- feature_matrix[-index,]

Criando e treinando o modelo de rede neural com 2 camadas escondidas, cada uma com 64 neurônios

In [42]:
nn <- neuralnet(score ~ ., data = train_split, hidden = c(64,64), 
                linear.output=FALSE, stepmax = 100, lifesign = "full")

hidden: 64, 64    thresh: 0.01    rep: 1/1    steps:      22	error: 0.25938	time: 2.91 secs


Avaliando o modelo treinado

In [43]:
test_X <- test_split[,1:2000]

In [44]:
test_Y <- test_split[,2001]

In [45]:
dim(test_split)

[1]  400 2001

In [46]:
prediction <- compute(nn, test_X)

In [47]:
head(cbind(test_Y, round(prediction$net.result, 4)))

,test_Y,
3,1,1.0000
7,1,0.8798
8,1,1.0000
9,1,0.1224
13,0,0.9966
28,1,0.9977


# Utilizando o modelo treinado com frases fora do corpus

In [48]:
texto <- "Esse produto é legal"

In [49]:
texto <- clean_string(texto)

In [50]:
print(texto)

[1] "esse produto e legal"


In [51]:
texto_features <- vectorize_sequences(texto, corpus_dfm, top_terms)

In [52]:
dim(texto_features)

[1]    1 2000

In [53]:
compute(nn, texto_features)$net.result

0.9865398
